In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time

In [2]:
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [3]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Import Metric Packages

In [4]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [5]:
mat_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

In [6]:
# Loading Mat file

In [7]:
data = loadmat('data/cardio.mat')

In [8]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [9]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# Checking shape of Independent Variables

In [10]:
type(data['X']) , data['X'].shape

(numpy.ndarray, (1831, 21))

# Checking shape of Dependent Variables

In [11]:
type(data['y']) , data['y'].shape

(numpy.ndarray, (1831, 1))

# Creating Data Frames

In [12]:
df_cols = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']

## ROC Performance evulotion table

In [13]:
roc_df = pd.DataFrame(columns=df_cols)

In [14]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


## precision n-scoresPerformance evulotion table

In [15]:
prn_df = pd.DataFrame(columns=df_cols)

In [16]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


## Time dataframe

In [17]:
time_df = pd.DataFrame(columns=df_cols)

In [18]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Exploring All Mat files

In [19]:
random_state = np.random.RandomState(42)

In [22]:
for file in mat_list:
    print("Processing: ", file + '...')
    mat = loadmat("data/"+file)
    
    X = mat['X']
    y = mat['y'].ravel() # Ravel flattens out the data
    
    outlier_fraction = np.count_nonzero(y)/len(y)
    outlier_percentage = round(outlier_fraction*100, ndigits = 4)
    
    # Construct containers for saving results
    roc_list = [file[:-4], X.shape[0], X.shape[1], outlier_percentage]
    prn_list = [file[:-4], X.shape[0], X.shape[1], outlier_percentage]
    time_list = [file[:-4], X.shape[0], X.shape[1], outlier_percentage]
    # file[:-4] keeps file name and removes .mat extension
    
    
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state = random_state)
    
    # Normalizing the data
    X_train_norm , X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outlier_fraction),
                   
                'Cluster-based Local Outlier Factor': CBLOF(
                    contamination=outlier_fraction, check_estimator=False, random_state=random_state),
        
                'Feature Bagging': FeatureBagging(contamination=outlier_fraction, random_state=random_state),
        
                'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outlier_fraction),
        
                'Isolation Forest': IForest(contamination=outlier_fraction, random_state=random_state),
        
                'K Nearest Neighbors (KNN)': KNN(contamination=outlier_fraction),
        
                'Local Outlier Factor (LOF)': LOF(contamination=outlier_fraction),
        
                'Minimum Covariance Determinant (MCD)': MCD(contamination=outlier_fraction, random_state=random_state),
        
                'One-class SVM (OCSVM)': OCSVM(contamination=outlier_fraction),
        
                'Principal Component Analysis (PCA)': PCA(contamination=outlier_fraction, random_state=random_state),
    }
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)        
        t1 = time()
        
        duration = round(t1-t0, ndigits = 5)
        
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print(clf_name,'ROC:',roc, 'precision @ rank n:',prn,'execution time:',duration,'s')

        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_cols
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_cols
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_cols
    prn_df = pd.concat([prn_df, temp_df], axis=0)

Processing:  arrhythmia.mat...
Angle-based Outlier Detector (ABOD) ROC: 0.7168 precision @ rank n: 0.3478 execution time: 0.2982 s
Cluster-based Local Outlier Factor ROC: 0.727 precision @ rank n: 0.3913 execution time: 0.21542 s
Feature Bagging ROC: 0.7241 precision @ rank n: 0.3478 execution time: 1.07715 s
Histogram-base Outlier Detection (HBOS) ROC: 0.7455 precision @ rank n: 0.4348 execution time: 0.18248 s
Isolation Forest ROC: 0.7548 precision @ rank n: 0.4348 execution time: 0.98642 s
K Nearest Neighbors (KNN) ROC: 0.7196 precision @ rank n: 0.3913 execution time: 0.28624 s
Local Outlier Factor (LOF) ROC: 0.7247 precision @ rank n: 0.3478 execution time: 0.15857 s


C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC: 0.7414 precision @ rank n: 0.3913 execution time: 1.0861 s
One-class SVM (OCSVM) ROC: 0.7132 precision @ rank n: 0.3913 execution time: 0.08876 s
Principal Component Analysis (PCA) ROC: 0.719 precision @ rank n: 0.3913 execution time: 0.09176 s
Processing:  cardio.mat...
Angle-based Outlier Detector (ABOD) ROC: 0.5301 precision @ rank n: 0.2241 execution time: 0.60837 s
Cluster-based Local Outlier Factor ROC: 0.7338 precision @ rank n: 0.2586 execution time: 0.21542 s
Feature Bagging ROC: 0.5945 precision @ rank n: 0.1552 execution time: 1.28657 s
Histogram-base Outlier Detection (HBOS) ROC: 0.8461 precision @ rank n: 0.431 execution time: 0.00997 s
Isolation Forest ROC: 0.9265 precision @ rank n: 0.4828 execution time: 0.61639 s
K Nearest Neighbors (KNN) ROC: 0.7034 precision @ rank n: 0.3448 execution time: 0.2364 s
Local Outlier Factor (LOF) ROC: 0.5584 precision @ rank n: 0.1379 execution time: 0.15562 s


C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-224.336525159365976 > -251.874766617635288). You may want to try with a higher value of support_fraction (current value: 0.510).
  RuntimeWarning)
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-153.817053364930416 > -220.545760387930471). You may want to try with a higher value of support_fraction (current value: 0.510).
  RuntimeWarning)
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: Runtime

Minimum Covariance Determinant (MCD) ROC: 0.8066 precision @ rank n: 0.4483 execution time: 0.87965 s
One-class SVM (OCSVM) ROC: 0.9444 precision @ rank n: 0.5517 execution time: 0.17054 s
Principal Component Analysis (PCA) ROC: 0.9611 precision @ rank n: 0.6379 execution time: 0.00997 s
Processing:  glass.mat...
Angle-based Outlier Detector (ABOD) ROC: 0.7222 precision @ rank n: 0.0 execution time: 0.06084 s
Cluster-based Local Outlier Factor ROC: 0.7917 precision @ rank n: 0.0 execution time: 0.07779 s
Feature Bagging ROC: 0.7593 precision @ rank n: 0.0 execution time: 0.05984 s
Histogram-base Outlier Detection (HBOS) ROC: 0.6667 precision @ rank n: 0.0 execution time: 0.00798 s
Isolation Forest ROC: 0.6944 precision @ rank n: 0.0 execution time: 0.45182 s
K Nearest Neighbors (KNN) ROC: 0.8009 precision @ rank n: 0.0 execution time: 0.013 s
Local Outlier Factor (LOF) ROC: 0.7315 precision @ rank n: 0.0 execution time: 0.00499 s
Minimum Covariance Determinant (MCD) ROC: 0.7546 precisi

...

C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.859190771539566 > -76.591352746989259). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.028289537302811 > -78.050981274347720). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.432170777806704 > -78.359194119587812). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\U

C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.516154178762157 > -77.633772205352969). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.701355796419179 > -76.693867101144676). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC: 0.9908 precision @ rank n: 0.7506 execution time: 20.59432 s
One-class SVM (OCSVM) ROC: 0.9917 precision @ rank n: 0.9568 execution time: 88.66297 s
Principal Component Analysis (PCA) ROC: 0.9897 precision @ rank n: 0.9511 execution time: 0.08777 s
Processing:  vertebral.mat...
Angle-based Outlier Detector (ABOD) ROC: 0.3611 precision @ rank n: 0.0909 execution time: 0.13165 s
Cluster-based Local Outlier Factor ROC: 0.34 precision @ rank n: 0.0909 execution time: 0.09076 s
Feature Bagging ROC: 0.4483 precision @ rank n: 0.0909 execution time: 0.09973 s
Histogram-base Outlier Detection (HBOS) ROC: 0.3263 precision @ rank n: 0.0 execution time: 0.00698 s
Isolation Forest ROC: 0.3462 precision @ rank n: 0.0909 execution time: 0.62036 s
K Nearest Neighbors (KNN) ROC: 0.396 precision @ rank n: 0.0 execution time: 0.02594 s
Local Outlier Factor (LOF) ROC: 0.4421 precision @ rank n: 0.0909 execution time: 0.00698 s


C:\Users\Sunil\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-22.820410996749565 > -41.107468354174515). You may want to try with a higher value of support_fraction (current value: 0.526).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC: 0.3948 precision @ rank n: 0.0 execution time: 0.22401 s
One-class SVM (OCSVM) ROC: 0.4645 precision @ rank n: 0.0909 execution time: 0.00698 s
Principal Component Analysis (PCA) ROC: 0.401 precision @ rank n: 0.0 execution time: 0.00299 s
Processing:  vowels.mat...
Angle-based Outlier Detector (ABOD) ROC: 0.98 precision @ rank n: 0.7368 execution time: 0.77593 s
Cluster-based Local Outlier Factor ROC: 0.8839 precision @ rank n: 0.2105 execution time: 0.31018 s
Feature Bagging ROC: 0.9317 precision @ rank n: 0.3158 execution time: 0.75099 s
Histogram-base Outlier Detection (HBOS) ROC: 0.7099 precision @ rank n: 0.1579 execution time: 0.01197 s
Isolation Forest ROC: 0.7404 precision @ rank n: 0.2632 execution time: 0.94747 s
K Nearest Neighbors (KNN) ROC: 0.9818 precision @ rank n: 0.6842 execution time: 0.12068 s
Local Outlier Factor (LOF) ROC: 0.92 precision @ rank n: 0.3158 execution time: 0.05485 s
Minimum Covariance Determinant (MCD) ROC: 0

In [23]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7676,0.7832,0.7708,0.8328,0.8226,0.7724,0.7708,0.8037,0.7884,0.7893
0,cardio,1831,21,9.6122,0.5477,0.8365,0.5931,0.8608,0.9309,0.7322,0.5784,0.778,0.9415,0.9612
0,glass,214,9,4.2056,0.3784,0.6216,0.6892,0.4324,0.3784,0.6351,0.7297,0.6351,0.1757,0.2838
0,ionosphere,351,33,35.8974,0.945,0.9181,0.9042,0.5475,0.8642,0.9642,0.9047,0.9761,0.885,0.8258
0,letter,1600,32,6.25,0.917,0.7576,0.8806,0.6206,0.5977,0.8885,0.8761,0.8438,0.5782,0.4995
0,lympho,148,18,4.0541,0.9608,1,1,1,1,1,1,0.9804,1,1
0,mnist,7603,100,9.2069,0.7736,0.8321,0.7293,0.5709,0.8128,0.848,0.7218,0.855,0.848,0.8475
0,arrhythmia,452,274,14.6018,0.7122,0.7552,0.7448,0.8096,0.8263,0.7612,0.7493,0.7564,0.7733,0.7699
0,cardio,1831,21,9.6122,0.6024,0.8301,0.5518,0.8674,0.9065,0.7479,0.5474,0.8529,0.9351,0.9516
0,glass,214,9,4.2056,0.6852,0.8287,0.7639,0.6991,0.6343,0.787,0.8148,0.7361,0.6759,0.6111


In [24]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3043,0.4783,0.4348,0.6087,0.5652,0.4783,0.4348,0.5652,0.4783,0.4783
0,cardio,1831,21,9.6122,0.2333,0.4833,0.1167,0.45,0.5,0.35,0.15,0.35,0.5333,0.65
0,glass,214,9,4.2056,0,0,0,0,0,0,0,0,0,0
0,ionosphere,351,33,35.8974,0.8333,0.8125,0.7292,0.375,0.6875,0.9167,0.7292,0.9167,0.7917,0.6667
0,letter,1600,32,6.25,0.3684,0.2105,0.3947,0.2105,0.0789,0.2632,0.3684,0.1842,0.1842,0.1053
0,lympho,148,18,4.0541,0,1,1,1,1,1,1,0,1,1
0,mnist,7603,100,9.2069,0.3586,0.3502,0.3376,0.097,0.3038,0.4219,0.3376,0.1941,0.3755,0.3755
0,arrhythmia,452,274,14.6018,0.32,0.36,0.36,0.52,0.48,0.32,0.32,0.44,0.36,0.36
0,cardio,1831,21,9.6122,0.2203,0.4237,0.1864,0.4915,0.4576,0.322,0.2034,0.5085,0.4915,0.5424
0,glass,214,9,4.2056,0,0,0,0,0,0,0,0,0,0


In [25]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,-25.4726,-6.54246,-1.15209,-4.25118,-1.19955,-0.1486,-0.15658,-4.64325,-0.12167,-0.21642
0,cardio,1831,21,9.6122,-0.65831,-0.26624,-1.42323,-0.01094,-0.65924,-0.2623,-0.15658,-1.00944,-0.18949,-0.09879
0,glass,214,9,4.2056,-0.06682,-0.07181,-0.05386,-0.00497,-0.47476,-0.01393,-0.00502,-0.10622,-0.00299,-0.00997
0,ionosphere,351,33,35.8974,-0.12165,-0.09776,-0.20445,-0.02094,-0.69115,-0.02793,-0.01197,-0.17151,-0.02094,-0.06183
0,letter,1600,32,6.25,-0.76595,-0.38398,-2.01162,-0.01599,-0.6393,-0.29621,-0.26828,-2.04853,-0.18949,-0.01097
0,lympho,148,18,4.0541,-0.05685,-0.10472,-0.08078,-0.01995,-0.70112,-0.01395,-0.00399,-0.06785,-0.00299,-0.00399
0,mnist,7603,100,9.2069,-13.5738,-2.78057,-87.0659,-0.08281,-3.73207,-11.0844,-10.473,-5.74186,-8.33671,-0.48048
0,arrhythmia,452,274,14.6018,0.26729,0.27626,1.2716,0.12267,1.13995,0.1506,0.13264,1.20378,0.13067,0.19349
0,cardio,1831,21,9.6122,0.71413,0.36303,1.7593,0.01197,0.87366,0.31116,0.17254,1.26562,0.17952,0.00998
0,glass,214,9,4.2056,0.08777,0.10572,0.07081,0.00598,0.54554,0.01399,0.00499,0.11484,0.00399,0.00598


## Predicting No. of Outliers and Inliers

In [26]:
N_samples = len(y)

n_inliers = int((1.0 - outlier_fraction) * N_samples)
n_outliers = N_samples - n_inliers

ground_truth = np.zeros(N_samples, dtype=int)
ground_truth[-n_outliers:] = 1

In [27]:
# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is', ground_truth.shape,'. Outlier are 1 and inliers are 0.\n')
print(ground_truth)

Number of inliers: 357
Number of outliers: 21
Ground truth shape is (378,) . Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1]


In [28]:
# Showing which model was which

for i, clf in enumerate(classifiers.keys()):
    print('Model', i + 1, clf)

Model 1 Angle-based Outlier Detector (ABOD)
Model 2 Cluster-based Local Outlier Factor
Model 3 Feature Bagging
Model 4 Histogram-base Outlier Detection (HBOS)
Model 5 Isolation Forest
Model 6 K Nearest Neighbors (KNN)
Model 7 Local Outlier Factor (LOF)
Model 8 Minimum Covariance Determinant (MCD)
Model 9 One-class SVM (OCSVM)
Model 10 Principal Component Analysis (PCA)
